In [189]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from ast import literal_eval
import os
tqdm.pandas()

In [190]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

# Introduction

All google spreadsheets listed below taken from [this](https://drive.google.com/drive/folders/1RqU2LT503Rcl6C-j8SnYVg92oTMj7S8H) gdrive link

At this stage have compiled: 

1. The tidied up GES information (`GES_dataGov_cleaned_23`)
2. The tidied up IGP information (`igp_tidy.xlsx`)
3. Mapping table for IGP-GES information (`IGP_GES_Mapping`)

## Objective

To create a single large table in tidy format containing: 
- University
- Faculty
- Unified Course Name
- Year
- Places
- Rank Points (requirement)
- GPA (requirement)
- Employment Rate Fields
- Earnings Fields
- *Category mapping? (For comparison between degrees, requires more thought)*

Need the table in **tidy format** because `ipyvizzu` *"stories"* can only reference a single fixed table of records defined at installation. Each story slide must use information from the original table. We can only change how the information is filtered.


## `IGP_GES_Mapping`
This is the table that maps IGP to GES information. 

In this workbook there are several sheets: 
1. `README`
    
    Background information on the workbook. 

2. University level tables

    Yearly mapping information for the unified name for each university's courses. 

    Currently the sheet names / universities considered are: 
    - `NUS`
    - `NTU`
    - `SMU`

3. `master_sheet`

    To map course names across the IGP & GES tables. 

    Table originally duplicated from the GES data. Records exist at the university-school-degree level. 

    The `mapping_name` field is the unified name, used to index the university-level sheets containing the mapping information. 
    
    For example considering *NUS Faculty of Arts and Social Sciences Bachelor of Arts / Hons*, the `mapping_name` *Arts* exists in the NUS and identifies the `Arts & Social Sciences` course for IGP data and `[Bachelor of Arts, Bachelor of Arts (Hons)
    ]` courses for GES data respectively. Refer to the README for more info. 
    
    Will need to use the `master_sheet` when aggregating individual university mapping tables to obtaint the university / faculty information. 

## Note

The spreadsheets available within this repo vs on the drive link may differ. Take the drive link version as the **most updated** since manual adjustment is more conveniently done on the table directly in gsheets. 

# Combination and Mapping

To combine the data across all Universities 

## Pre-Processing

In [191]:
# Create ExcelFile object
mappping_spreadsheet = pd.ExcelFile("../../data/IGP_GES_Mapping_Table.xlsx")

In [192]:
# All the sheet names
print(mappping_spreadsheet.sheet_names)

['README', 'NUS', 'NTU', 'SMU', 'Master Sheet']


In [193]:
# Instantiate master dataframe
master_df_raw = pd.DataFrame()

# Combine into one dataframe
for uni in tqdm(['NUS', 'NTU', 'SMU']):
    # Read in that university's table
    tmp_df = pd.read_excel(mappping_spreadsheet, sheet_name=uni)
    # Add university name (in short form)
    tmp_df['uni']=uni
    # Append to master dataframe
    master_df_raw=pd.concat([master_df_raw, tmp_df], ignore_index=True)


100%|██████████| 3/3 [00:00<00:00, 49.78it/s]


In [194]:
# Add the long form version of the uni name
master_df_raw['university'] = master_df_raw['uni'].replace({
    'NUS': 'National University of Singapore'
    , 'NTU': 'Nanyang Technological University'
    , 'SMU': 'Singapore Management University'
})

In [195]:
master_df_raw.head()

,select_course,year,igp_course,ges_course,uni,university
0,Arts,2013,Arts & Social Sciences,Bachelor of Arts,NUS,National University of Singapore
1,Arts,2014,Arts & Social Sciences,Bachelor of Arts,NUS,National University of Singapore
2,Arts,2015,Arts & Social Sciences,Bachelor of Arts,NUS,National University of Singapore
3,Arts,2016,Arts & Social Sciences,Bachelor of Arts,NUS,National University of Singapore
4,Arts,2017,Arts & Social Sciences,Bachelor of Arts,NUS,National University of Singapore


In [196]:
# Trim whitespace from relevant text columns
text_cols=['select_course', 'igp_course', 'ges_course', 'uni', 'university']
for col in text_cols:
    master_df_raw[col]=master_df_raw[col].str.strip()

## Obtaining School

Obtaining the School (Faculty) each degree falls under by mapping back to the `master_sheet` table

In [197]:
# Read in master sheet
master_sheet = pd.read_excel(mappping_spreadsheet, sheet_name='Master Sheet')

# Strip leading & trailing whitespace
text_cols = text_cols=['university', 'school', 'degree', 'mapping_name', 'Notes']
for col in text_cols:
    master_sheet[col]=master_sheet[col].str.strip()

In [198]:
########
# obtain the School associated with each degree by joining on the university name & mapping_name
########
master_df_school = master_df_raw.merge(
                    master_sheet[['university', 'mapping_name', 'school']]
                    ,left_on=('university', 'select_course')
                    , right_on=('university', 'mapping_name')
                    , how='left')

In [199]:
# There should be no missing school values
tmp=master_df_school.loc[master_df_school.school.isna()]
print(tmp.shape[0])
# Empty Table

0


In [200]:
# Reorder & Keep columns
master_df_school=master_df_school[['uni', 'school', 'select_course', 'year'
                                   , 'igp_course', 'ges_course', 'university']]
# rename column 
master_df_school.rename(columns={
    'select_course':'course'
    }, inplace=True)


In [201]:
print(f"Before joining there were {master_df_raw.shape[0]:,} rows.")
print(f"Now there were {master_df_school.shape[0]:,} rows.")

Before joining there were 889 rows.
Now there were 889 rows.


# Obtaining IGP Information

Filling in the IGP dimensions for our dataset. Will require joining to the `igp_tidy.xlsx` table. 

We want to extract the fields: 
- Places
- GPA
- RP


In [202]:
# Read in igp_tidy.xlsx
igp_df = pd.read_excel("../../data/igp_tidy.xlsx")
# Rename columns for easier joining
igp_df.rename(columns={
                    'University': 'uni'
                    , 'Course': 'course_IGP'
                }
                , inplace=True)

# Strip leading & trailing whitespace but only for selected cols
coltypes = igp_df.dtypes
text_cols=['Faculty', 'course_IGP', 'AY', 'uni', 'RP']
for col in text_cols:
    igp_df[col]=igp_df[col].str.strip()


In [203]:
# join on course name and university
master_df_igp = master_df_school.merge(
                igp_df[['uni', 'course_IGP', 'year', 'Places', 'GPA', 'RP']]
                , left_on=('uni', 'igp_course', 'year')
                , right_on=('uni', 'course_IGP', 'year')
                , how='left'
)

In [204]:
print(f"There were {master_df_school.shape[0]:,} rows before merging")
# There were 900 rows before merging
print(f"There are {master_df_igp.shape[0]:,} rows after merging")
# There are 900 rows after merging

There were 889 rows before merging
There are 889 rows after merging


In [205]:
# Any entries where there was no IGP match?
missing=master_df_igp.loc[master_df_igp[['Places', 'GPA', 'RP']].isna().all(axis=1)]
missing
# No entries

,uni,school,course,year,igp_course,ges_course,university,course_IGP,Places,GPA,RP
677,NTU,National Institute of Education (NIE),Science (Education),2014,NaN,Science (Education),Nanyang Technological University,NaN,NaN,NaN,NaN
688,NTU,National Institute of Education (NIE),Arts (Education),2014,NaN,Arts (Education),Nanyang Technological University,NaN,NaN,NaN,NaN


## Places Adjustment

For some courses the `Places` value for a particular year may not be a number. 

Instead it redirects to a separate course where the places are shared. For example the `Arts` course in NUS from 2021 onwards shares a common placement intake under *Humanities & Sciences* with the `Science` & `Data Science & Analytics` intakes. 

However this is already reflected in the mapping table (`master_df_raw`), the *generic* IGP course mapped to that specific course is the *general* intake. For example all science / specific arts courses from 2021 onwards have their IGP requirements and places available tagged to a general `"Humanities & Sciences" intake`. 

The case we investigate is when only **places** are shared with other courses, but the specific course has its own Rank Points / GPA critiera.

In [206]:
# get the full set of courses where the places are special cases (and are not missing)
master_df_igp.loc[(~master_df_igp.Places.astype(str).str.isnumeric())
                  & (~master_df_igp.Places.isna())
                  , ['uni', 'school', 'course', 'Places']].drop_duplicates()

,uni,school,course,Places
186,NUS,Faculty of Science,Data Science and Analytics,Science


There is only **one** such case. 

For this case, will draw the *Places* values from the course *Science* in the IGP data. For context, *Data Science & Analytice* shared its place information with the general Science intake in 2016 & 2017.

In [207]:
def extract_places(row):
    course = row['Places']
    year = row['year']
    uni = row['uni']
    value = igp_df.loc[(igp_df.course_IGP==course) 
                        & (igp_df.year==year)
                        & (igp_df.uni==uni), 'Places'].item()
    return value

In [208]:
tmp= master_df_igp.loc[(~master_df_igp.Places.astype(str).str.isnumeric())
                  & (~master_df_igp.Places.isna())
                  ]
tmp

,uni,school,course,year,igp_course,ges_course,university,course_IGP,Places,GPA,RP
186,NUS,Faculty of Science,Data Science and Analytics,2016,Data Science & Analytics,NaN,National University of Singapore,Data Science & Analytics,Science,NaN,AAB/C (80)
187,NUS,Faculty of Science,Data Science and Analytics,2017,Data Science & Analytics,NaN,National University of Singapore,Data Science & Analytics,Science,NaN,AAA/C (82.5)
188,NUS,Faculty of Science,Data Science and Analytics,2018,Data Science & Analytics,NaN,National University of Singapore,Data Science & Analytics,Science,NaN,AAB/C (80)


In [209]:
# Extract for the relevant rows
master_df_igp.loc[(~master_df_igp.Places.astype(str).str.isnumeric())
                  & (~master_df_igp.Places.isna())
                  , 'Places'] = master_df_igp.loc[\
                    (~master_df_igp.Places.astype(str).str.isnumeric())
                    & (~master_df_igp.Places.isna())].apply(extract_places, axis=1)

In [210]:
master_df_igp.loc[master_df_igp.course=='Data Science and Analytics']

,uni,school,course,year,igp_course,ges_course,university,course_IGP,Places,GPA,RP
186,NUS,Faculty of Science,Data Science and Analytics,2016,Data Science & Analytics,NaN,National University of Singapore,Data Science & Analytics,1027,NaN,AAB/C (80)
187,NUS,Faculty of Science,Data Science and Analytics,2017,Data Science & Analytics,NaN,National University of Singapore,Data Science & Analytics,851,NaN,AAA/C (82.5)
188,NUS,Faculty of Science,Data Science and Analytics,2018,Data Science & Analytics,NaN,National University of Singapore,Data Science & Analytics,927,NaN,AAB/C (80)
189,NUS,Faculty of Science,Data Science and Analytics,2019,Data Science & Analytics,NaN,National University of Singapore,Data Science & Analytics,179,NaN,AAB/C (80)
190,NUS,Faculty of Science,Data Science and Analytics,2020,Data Science & Analytics,Data Science and Analytics,National University of Singapore,Data Science & Analytics,204,NaN,AAA/C (82.5)
191,NUS,Faculty of Science,Data Science and Analytics,2021,Humanities & Sciences,Data Science and Analytics,National University of Singapore,Humanities & Sciences,2100,3.79,ABB/B (78.75)
192,NUS,Faculty of Science,Data Science and Analytics,2022,Humanities & Sciences,Data Science and Analytics,National University of Singapore,Humanities & Sciences,1971,3.67,ABB/C (77.5)
193,NUS,Faculty of Science,Data Science and Analytics,2023,Humanities & Sciences,Data Science and Analytics,National University of Singapore,Humanities & Sciences,1949,3.62,ABB/B (78.75)


## GPA & Rank Points

Checking that the data types for `GPA` & `RP` (Rank Points) are numerical. 

For `RP`, need to extract just the score without the letter grade breakdown before converting. 

### GPA

In [211]:
master_df_igp[['GPA', 'RP']].dtypes

GPA    object
RP     object
dtype: object

In [212]:
# What non-float values are there in the GPA column?
master_df_igp.loc[~master_df_igp['GPA'].apply(np.isreal), 'GPA'].unique()

array(['#'], dtype=object)

There are no malformed float objects. Hence we can convert the column directly with `pd.to_numeric` and force the unconvertible values (including the `'#'` character) to missing `np.NaN` values. 

In [213]:
master_df_igp['GPA'] = pd.to_numeric(master_df_igp['GPA'], errors='coerce')

### RP 

In [214]:
master_df_igp['RP']=master_df_igp['RP'].str.extract(r"(?<=\()([0-9\.]+)(?=\))")


In [215]:
master_df_igp.loc[master_df_igp.RP.isna()]

,uni,school,course,year,igp_course,ges_course,university,course_IGP,Places,GPA,RP
677,NTU,National Institute of Education (NIE),Science (Education),2014,NaN,Science (Education),Nanyang Technological University,NaN,NaN,NaN,NaN
688,NTU,National Institute of Education (NIE),Arts (Education),2014,NaN,Arts (Education),Nanyang Technological University,NaN,NaN,NaN,NaN


Only two missing entries did not have any valid IGP information. Hence the extraction was successful. 

In [216]:
# convert the datatype
master_df_igp['RP']=master_df_igp.RP.astype(float)

In [217]:
master_df_igp[['GPA', 'RP']].dtypes

GPA    float64
RP     float64
dtype: object

Conversion Successful!

# Obtaining GES information

Now to obtain the GES information for each row. 

In [218]:
# Read in the cleaned up GES information
ges_df = pd.read_excel("../../data/GES_dataGov_cleaned_23.xlsx", sheet_name=0)
# Filter for in-scope universities
ges_df=ges_df.loc[ges_df.university.isin(
    [
    'Nanyang Technological University',
    'National University of Singapore',
    'Singapore Management University', ]
    )]
# Add shorthand uni name column
ges_df['uni'] = ges_df['university'].replace(
    {
        'Nanyang Technological University' : 'NTU',
        'National University of Singapore' : 'NUS',
        'Singapore Management University' : 'SMU'
    })

In [219]:
ges_df.columns.tolist()

['year',
 'university',
 'school',
 'degree',
 'employment_rate_overall',
 'employment_rate_ft_perm',
 'basic_monthly_mean',
 'basic_monthly_median',
 'gross_monthly_mean',
 'gross_monthly_median',
 'gross_mthly_25_percentile',
 'gross_mthly_75_percentile',
 'include',
 'uni']

In [220]:
ges_df.head(2)

,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile,include,uni
0,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy and Business,97.4,96.1,3701,3200,3727,3350,2900,4000,1,NTU
1,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy,97.1,95.7,2850,2700,2938,2700,2700,2900,1,NTU


In [221]:
# Perform the joining
ges_stats = ['employment_rate_overall',
             'employment_rate_ft_perm', 
             'basic_monthly_mean', 
             'basic_monthly_median', 
             'gross_monthly_mean', 
             'gross_monthly_median', 
             'gross_mthly_25_percentile', 
             'gross_mthly_75_percentile']

In [222]:
# Perform the merge. 'ges_course' in the mapping dataframe is the 'degree' name in 
# the GES table
master_df_ges = master_df_igp.merge(ges_df[['uni', 'year', 'degree'] + ges_stats]
                                    , left_on=('uni', 'year', 'ges_course')
                                    , right_on = ('uni', 'year', 'degree')
                                    , how='left')

In [223]:
print(f"There were {master_df_igp.shape[0]:,} rows before merging")
print(f"After merging there are {master_df_ges.shape[0]:,} rows")

There were 889 rows before merging
After merging there are 889 rows


In [225]:
master_df_ges.loc[(master_df_ges.degree.isna())
                  & (~master_df_ges.ges_course.isna())].groupby(['uni', 'course']).size()

Series([], dtype: int64)

Empty series. This means all the GES mappings are accounted for. Rows with GES information missing have no `ges_course` for that row, meaning there was no GES information available for that course that year. 

In [230]:
# convert the dtypes
for col in ges_stats:
    master_df_ges[col] = master_df_ges[col].astype(float)

In [231]:
master_df_ges[ges_stats].dtypes

employment_rate_overall      float64
employment_rate_ft_perm      float64
basic_monthly_mean           float64
basic_monthly_median         float64
gross_monthly_mean           float64
gross_monthly_median         float64
gross_mthly_25_percentile    float64
gross_mthly_75_percentile    float64
dtype: object

In [235]:
# Reorganize columns
colorder = ['uni', 'school', 'course', 'year', 'Places', 'GPA', 'RP'] + ges_stats + ['university']
master_df_final = master_df_ges[colorder]

In [236]:
master_df_final.head(1)

,uni,school,course,year,Places,GPA,RP,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile,university
0,NUS,Faculty of Arts and Social Sciences,Arts,2013,1562,3.68,77.5,84.8,70.1,2741.0,2730.0,2888.0,2800.0,2500.0,3080.0,National University of Singapore


# Special GES treatment

## NUS Data

Pre-processing NUS data. 

For degrees with multiple GES results (e.g. *Nursing* has the GES records for Honours / **No** Honours), we take the the path <mark>with Honours only</mark>.



In [7]:
# NUS Data
nus_df=pd.read_excel(mappping_spreadsheet, sheet_name='NUS')

In [8]:
# Courses to drop. These all have the hounours version
drop_courses = [
    'Arts'
    , 'Science'
    , 'Accountancy'
    , 'Business Administration'
    , 'Nursing'
]

In [9]:
nus_df.head()

,select_course,year,igp_course,ges_course
0,Arts,2013,Arts & Social Sciences,Bachelor of Arts
1,Arts,2014,Arts & Social Sciences,Bachelor of Arts
2,Arts,2015,Arts & Social Sciences,Bachelor of Arts
3,Arts,2016,Arts & Social Sciences,Bachelor of Arts
4,Arts,2017,Arts & Social Sciences,Bachelor of Arts


In [10]:
# Print Course Count & Row Count
print(f"Initially there are {nus_df.select_course.nunique():,} unique courses"\
      f"and {nus_df.shape[0]:,} rows. ")
# Initially there are 35 unique coursesand 379 rows. 

# Remove the specified courses
nus_df = nus_df.loc[~nus_df.select_course.isin(drop_courses)]

# Re print Course Count & Row Count
print(f"After removal of specified courses there"\
      f"are {nus_df.select_course.nunique():,} unique courses"\
      f"and {nus_df.shape[0]:,} rows. ")
# After removal of specified courses thereare 30 unique coursesand 324 rows. 

Initially there are 35 unique coursesand 379 rows. 
After removal of specified courses thereare 30 unique coursesand 324 rows. 


In [11]:
# Specify University
nus_df['university']='National University of Singapore'

## NTU Data

No need to make any adjustments to this dataset since each degree corresponds to one GES outcome only. 

Add university name only. 

In [13]:
ntu_df = pd.read_excel(mappping_spreadsheet, sheet_name='NTU')

In [14]:
# Specify University
ntu_df['university']='Nanyang Technological University'